STEP 1: Highlight the imported packages and libaries

In [ ]:
# importing all the necessary libaries and packages

import numpy as np
import pandas as pd

import math
import itertools
import time
import random

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

import seaborn as sns

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

STEP 2 and 3: Loading the data set using Pytorch

In [ ]:
'''
# This loads the MNIST training data using the PyTorch datasets module, setting the root parameter to ROOT, and train parameter to True to indicate that the data is for training. 
If the data has not already been downloaded, it will be downloaded using the download parameter set to True.
it then calculates the statistical representation of the data, which is the mean and standard deviation of the pixel values in the training data. 
This is done by first converting the data to float type and then dividing it by 255 to scale the pixel values between 0 and 1. 
The mean and standard deviation are then calculated using the PyTorch data module. 

'''

train_data = torchvision.datasets.EMNIST(root = './data',  split='balanced', download = True,
                                            train = True, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4,), (0.4,))]))
test_data = torchvision.datasets.EMNIST(root = './data', split='balanced', download=True,
                                            train=False, transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4,), (0.4,))]))

print("number of training dataset:", len(train_data))
print("number of testing dataset:", len(test_data ))

In [ ]:
'''
used to determine the proportion of the data that will be used for validation. In this case, it is set to 0.8, 
which means that 80% of the data will be used for training and 20% will be used for validation.
'''

#splitting data into traning and validation
VALID_RATIO = 0.8 
n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples

'''
used to randomly split the training data into two subsets, train_data and valid_data, using the number of 
examples for each set that was calculated earlier.
'''
train_data, valid_data = data.random_split(train_data,
                                           [n_train_examples, n_valid_examples])

#printing out the number of examples in each set.
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
#creating  data loaders for the training and testing sets
#defining the batche size of 250 samples, with the data being suffeled  
batch_size = 250

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

STEP 4:  Understanding the dataset and visualising the dataset

In [ ]:
# visualising the EMNIST dataset images
classes = train_data.dataset.classes
num_classes = len(classes)

# a function to convert label index to class name
def convert_label_to_classname(label):
    return classes[label]

# getting a batch of training data
images, labels = next(iter(train_loader))

# plotting the images from the data set (we are plotting 4)
fig = plt.figure(figsize=(10, 10))
for i in range(min(batch_size, 4)):
    ax = fig.add_subplot(4, 4, i+1)
    ax.imshow(images[i].squeeze().numpy(), cmap='gray')
    ax.set_title(convert_label_to_classname(labels[i].item()))
    ax.axis('off')
plt.tight_layout()
plt.show()


STEP 5: Hyperparameters 

In [ ]:
#if a GPU systems is avalible it will be used (used for faster computation) if not then it will use CPU
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [ ]:
# defining the models parameters
# The image size set up is 784 because that how many pixels there are in total (28*28=784)
input_size = 784
# The Number of nodes at hidden layer
hidden_size = 100
# number of output classes discrete range [0,9]
num_classes = 47
batch_size = 64
lr = 0.01
epochs = 20
new_epoch = ...
activation = 'relu'
optimiser = optim.SGD
batch_norm = True
regularization = 0.001
dropout = 0.2

Multi Layer Perceptron (MLP)

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, activation, batch_norm, dropout, regularization):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, num_classes)
        self.activation = activation
        self.batch_norm = batch_norm
        self.dropout = dropout
        self.regularization = regularization
        if self.batch_norm:
            self.bn1 = nn.BatchNorm1d(hidden_size)
            self.bn2 = nn.BatchNorm1d(hidden_size)
            self.bn3 = nn.BatchNorm1d(hidden_size)
        if self.regularization:
            self.l1 = nn.L1Loss()
            self.l2 = nn.MSELoss()
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.fc1(x)
        if self.batch_norm:
            out = self.bn1(out)
        if self.activation == 'relu':
            out = F.relu(out)
        elif self.activation == 'sigmoid':
            out = F.sigmoid(out)
        elif self.activation == 'tanh':
            out = F.tanh(out)
        if self.dropout:
            out = F.dropout(out, p=self.dropout)
        out = self.fc2(out)
        if self.batch_norm:
            out = self.bn2(out)
        if self.activation == 'relu':
            out = F.relu(out)
        elif self.activation == 'sigmoid':
            out = F.sigmoid(out)
        elif self.activation == 'tanh':
            out = F.tanh(out)
        if self.dropout:
            out = F.dropout(out, p=self.dropout)
        out = self.fc3(out)
        if self.batch_norm:
            out = self.bn3(out)
        if self.activation == 'relu':
            out = F.relu(out)
        elif self.activation == 'sigmoid':
            out = F.sigmoid(out)
        elif self.activation == 'tanh':
            out = F.tanh(out)
        if self.dropout:
            out = F.dropout(out, p=self.dropout)
        out = self.fc4(out)
        return out

In [ ]:
# defining the MLP model
model = MLP(input_size, hidden_size, num_classes, activation, batch_norm, dropout, regularization).to(device)

# defining the loss function, optimiser and learning rate (lr) scheduler
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode='min', factor=0.1, patience=1, verbose=True)

accuracy, losses, epochs = [], [], epochs

# training the model
start_time = time.time()
for epoch in range(epochs):
    correct, total = 0, 0
    for images, labels in train_loader:
        # Move the images and labels to the device
        images, labels = images.to(device), labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # regularisation
        if regularization in ('l1', 'l2'):
            reg_loss = sum(torch.norm(param, p=int(regularization[1])) for param in model.parameters())
            loss += regularization * reg_loss
        # Backward and optimize
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        # Compute training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    scheduler.step(loss)

    # computing the training accuracy for epoch
    accuracy.append(correct / total)
    losses.append(loss.item())

    # printing the loss and accuracy after each epoch
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Training Accuracy: {accuracy[-1] * 100:.2f}%")

training_time = time.time() - start_time
print(f"Training time: {training_time:.2f}s")

result = list(zip(range(epochs), accuracy, losses))


In [ ]:
def hyper_params_test(activation, opt, batch_norm, regularization, dropout, lr_scheduler, data): 
    
    model = MLP(input_size, hidden_size, num_classes, activation, batch_norm, dropout, regularization).to(device)
    
    # defining the loss function, optimiser, and learning rate scheduler
    criterion = nn.CrossEntropyLoss()
    if opt == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    elif opt == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.001)
    elif opt == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)

    if lr_scheduler == 'ReduceLROnPlateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
    elif lr_scheduler == 'StepLR':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # training the model with initialiseed empty lists to store the accuracy, losses and itter
    accuracy = []
    losses = []
    iter = []
    startTime = time.time()
    for epoch in range(epochs):
        correct = 0
        total = 0
        for images, labels in data:
            # Move the images and labels to the device
            images = images.to(device)
            labels = labels.to(device)

             # forward pass to obtain predicted output
            outputs = model(images)
            loss = criterion(outputs, labels)
            if regularization == 'l1':
                l1_regularization = torch.tensor(0, dtype=torch.float32)
                for param in model.parameters():
                    l1_regularization += torch.norm(param, 1)
                loss += 0.001 * l1_regularization
            elif regularization == 'l2':
                l2_regularization = torch.tensor(0, dtype=torch.float32)
                for param in model.parameters():
                    l2_regularization += torch.norm(param, 2)
                loss += 0.01 * l2_regularization
            else:
                pass 

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # computing training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        scheduler.step(loss)

        # computing training accuracy for the epoch
        Accuracy = correct / total
        trainingAccuracy = 100 * Accuracy

        # printing the loss and accuracy after each epoch
#         print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Training Accuracy: {trainingAccuracy:.2f}%")

        accuracy.append(Accuracy)  # updating the list for storing accuracy values after 
        losses.append(loss.item())  # updating the list for storing loss values
        iter.append(epoch+1)
    
    result = list(zip(iter, accuracy, losses))  # update the variable names in the zip function

    endTime = time.time()
    trainingTime = endTime - startTime
#     print(f"Training time: {trainingTime}")
    return result

In [ ]:
train_result = hyper_params_test('relu', 'Adam', True, 'l1', 0.2, 'ReduceLROnPlateau', train_loader)

train_result

In [ ]:
val_result = hyper_params_test('relu', 'Adam', True, 'l1', 0.2, 'ReduceLROnPlateau', val_loader)

val_result

In [ ]:
def random_search_hyperparams(num_trials, data):
    startTime = time.time()
    # defining the range of values for each hyperparameter used in the MLP traning
    hyperparams = {
        'activation': ['relu', 'sigmoid', 'tanh'], 
        'opt': ['SGD', 'Adam', 'RMSprop'],
        'batch_norm': [True, False],
        'regularization': ['l1', 'l2', 'none'],
        'dropout': [0.0, 0.2],
        'scheduler': ['ReduceLROnPlateau', 'StepLR']
    }

    best_accuracy = 0.0
    best_hyperparams = None

    for i in range(num_trials):
        # Sample a set of hyperparameters at random
        activation = random.choice(hyperparams['activation'])
        opt = random.choice(hyperparams['opt'])
        batch_norm = random.choice(hyperparams['batch_norm'])
        regularization = random.choice(hyperparams['regularization'])
        dropout = random.uniform(*hyperparams['dropout'])
        scheduler = random.choice(hyperparams['scheduler'])

        # Test the current set of hyperparameters
        #result = hyper_params_test(activation, opt, batch_norm, regularization, dropout, scheduler, data)

        # Compute the accuracy of the model for the current set of hyperparameters
        accuracy = result[-1][1]

        # Check if the current set of hyperparameters has produced the best accuracy so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparams = {
                'activation': activation,
                'opt': opt,
                'batch_norm': batch_norm,
                'regularization': regularization,
                'dropout': dropout,
                'scheduler': scheduler
            }
    endTime = time.time()
    trainingTime = endTime - startTime
    return best_hyperparams, best_accuracy, trainingTime

In [ ]:
best_hyperparams, best_accuracy, trainingTime = random_search_hyperparams(3, train_loader)

In [ ]:
trainingTime

In [ ]:
best_hyperparams

In [ ]:
best_accuracy

In [ ]:
best_hyperparams = best_hyperparams
best_result = hyper_params_test(best_hyperparams['activation'], best_hyperparams['opt'], best_hyperparams['batch_norm'], best_hyperparams['regularization'], best_hyperparams['dropout'], best_hyperparams['scheduler'], train_loader)

In [ ]:
# getting the first two items in each tuple and store in separate lists
train_iter = [x[0] for x in best_result]
train_accuracy = [x[1] for x in best_result]
val_iter = [x[0] for x in val_result]
val_accuracy = [x[1] for x in val_result]

# plotting a graph for both training and validation 
plt.plot(train_iter, train_accuracy, '-o', label='Training')
plt.plot(val_iter, val_accuracy, '-o', label ='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy graph')
plt.legend()
plt.show()

In [ ]:
# Get the first two items in each tuple and store in separate lists
train_iter = [x[0] for x in best_result]
train_accuracy = [x[2] for x in best_result]

# Plot elbow graph for both training and validation loss
plt.plot(train_iter, train_accuracy, '-o', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss function Plot')
plt.legend()
plt.show()

In [ ]:
# Save the model state dictionary
torch.save(model.state_dict(), 'mlp_model.pth')

In [ ]:
trainingTime

Convelutional Neural Network (CNN)

In [ ]:
#defining the CNN model
class CNN(nn.Module):
    def __init__(self, input_size, num_classes, activation, batch_norm, dropout, regularization):
        super(CNN, self).__init__()
        # convolutional layer 1 and max pool layer 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # convolutional layer 2
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.MaxPool2d(kernel_size=2) 
        # fully connected layers and max pool layer 1
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=47)
        self.activation = activation
        self.batch_norm = batch_norm
        self.dropout = dropout
        self.regularization = regularization
        if self.batch_norm:
            self.bn1 = nn.BatchNorm2d(6)
            self.bn2 = nn.BatchNorm2d(16)
        if self.regularization:
            self.l1 = nn.L1Loss()
            self.l2 = nn.MSELoss()

    def forward(self, x):
       # convolutional layer 1
        out = self.conv1(x)
        if self.batch_norm:
            out = self.bn1(out)
        if self.activation == 'relu':
            out = F.relu(out)
        elif self.activation == 'sigmoid':
            out = F.sigmoid(out)
        elif self.activation == 'tanh':
            out = F.tanh(out)
        # max pool layer 1
        out = self.pool(out)
        # convolutional layer 2
        out = self.conv2(out)
        if self.batch_norm:
            out = self.bn2(out)
        if self.activation == 'relu':
            out = F.relu(out)
        elif self.activation == 'sigmoid':
            out = F.sigmoid(out)
        elif self.activation == 'tanh':
            out = F.tanh(out)
        # max pool layer 2
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        if self.activation == 'relu':
            out = F.relu(out)
        elif self.activation == 'sigmoid':
            out = F.sigmoid(out)
        elif self.activation == 'tanh':
            out = F.tanh(out)
        if self.dropout:
            out = F.dropout(out, p=self.dropout)
        out = self.fc2(out)
        return out

In [ ]:
# inililising the CNN model for the traning function
model = CNN(input_size, num_classes, activation, batch_norm, dropout, regularization).to(device)

# defining the optimiser, learning rate (lr) and loss function
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode='min', factor=0.1, patience=1, verbose=True)

accuracy, losses, epochs = [], [], epochs

# Training the model
start_time = time.time()
for epoch in range(epochs):
    correct, total = 0, 0
    for images, labels in train_loader:
        # Move the images and labels to the device
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # regularisation
        if regularization in ('l1', 'l2'):
            reg_loss = sum(torch.norm(param, p=int(regularization[1])) for param in model.parameters())
            loss += regularization * reg_loss

        # Backward and optimize
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        # Compute training accuracy
        _, predicted = torch.max(outputs.data, 1)
        # the total number of labels
        total += labels.size(0)
        # the total correct predictions
        correct += (predicted == labels).sum().item()

    scheduler.step(loss)

    # Compute training accuracy for the epoch
    accuracy.append(correct / total)
    losses.append(loss.item())

    # printing the loss and accuracy after each epoch 
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Training Accuracy: {accuracy[-1] * 100:.2f}%")

training_time = time.time() - start_time
print(f"Training time: {training_time:.2f}s")

result = list(zip(range(epochs), accuracy, losses))


In [ ]:
def hyper_params_test(activation, opt, batch_norm, regularization, dropout, lr_scheduler, data): 
    
    model = CNN(input_size, num_classes, activation, batch_norm, dropout, regularization).to(device)
    
    # Define the loss function, optimizer, and learning rate scheduler
    criterion = nn.CrossEntropyLoss()
    if opt == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    elif opt == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.001)
    elif opt == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)

    if lr_scheduler == 'ReduceLROnPlateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
    elif lr_scheduler == 'StepLR':
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # training the CNN model
    # initialise empty lists to store the accuracies, losses, iter 
    accuracy = []
    losses = []
    iter = []
    startTime = time.time()
    # loop over the number of epochs specified
    for epoch in range(epochs):
        correct = 0
        total = 0
        for images, labels in data:
            # Move the images and labels to the device
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            if regularization == 'l1':
                l1_regularization = torch.tensor(0, dtype=torch.float32)
                for param in model.parameters():
                    l1_regularization += torch.norm(param, 1)
                loss += 0.001 * l1_regularization
            elif regularization == 'l2':
                l2_regularization = torch.tensor(0, dtype=torch.float32)
                for param in model.parameters():
                    l2_regularization += torch.norm(param, 2)
                loss += 0.01 * l2_regularization
            else:
                pass  # Do nothing if regularization is None

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        scheduler.step(loss)

        # Compute training accuracy for the epoch
        Accuracy = correct / total
        trainingAccuracy = 100 * Accuracy

        # Print the loss and accuracy after each epoch
#         print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Training Accuracy: {trainingAccuracy:.2f}%")

        accuracy.append(Accuracy)  # Update the list for storing accuracy values
        losses.append(loss.item())  # Update the list for storing loss values
        iter.append(epoch+1)
    
    result = list(zip(iter, accuracy, losses))  # Update variable name in the zip function

    endTime = time.time()
    trainingTime = endTime - startTime
#     print(f"Training time: {trainingTime}")
    return result

In [ ]:
train_result = hyper_params_test('relu', 'Adam', True, 'l1', 0.2, 'ReduceLROnPlateau', train_loader)
train_result

In [ ]:
val_result = hyper_params_test('relu', 'Adam', True, 'l1', 0.2, 'ReduceLROnPlateau', val_loader)
val_result

In [ ]:
def random_search_hyperparams(num_trials, data):
    startTime = time.time()
    # defining the range of values for each of the hyperparameter
    hyperparams = {
        'activation': ['relu', 'sigmoid', 'tanh'], 
        'opt': ['SGD', 'Adam', 'RMSprop'],
        'batch_norm': [True, False],
        'regularization': ['l1', 'l2', 'none'],
        'dropout': [0.0, 0.2],
        'scheduler': ['ReduceLROnPlateau', 'StepLR']
    }

    best_accuracy = 0.0
    best_hyperparams = None

    for i in range(num_trials):
        # sample a set of hyperparameters at random to start
        activation = random.choice(hyperparams['activation'])
        opt = random.choice(hyperparams['opt'])
        batch_norm = random.choice(hyperparams['batch_norm'])
        regularization = random.choice(hyperparams['regularization'])
        dropout = random.uniform(*hyperparams['dropout'])
        scheduler = random.choice(hyperparams['scheduler'])

        # Test the current set of hyperparameters
        result = hyper_params_test(activation, opt, batch_norm, regularization, dropout, scheduler, data)

        # Compute the accuracy of the model for the current set of hyperparameters
        accuracy = result[-1][1]

        # Check if the current set of hyperparameters has produced the best accuracy so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparams = {
                'activation': activation,
                'opt': opt,
                'batch_norm': batch_norm,
                'regularization': regularization,
                'dropout': dropout,
                'scheduler': scheduler
            }
    endTime = time.time()
    trainingTime = endTime - startTime
    return best_hyperparams, best_accuracy, trainingTime

In [ ]:
best_hyperparams, best_accuracy, trainingTime = random_search_hyperparams(3, train_loader)

In [ ]:
trainingTime

In [ ]:
best_hyperparams

In [ ]:
best_accuracy

In [ ]:
#storing the first two items in each tuplein a separate lists
train_iter = [x[0] for x in best_result]
train_accuracy = [x[1] for x in best_result]
val_iter = [y[0] for y in val_result]
val_accuracy = [y[1] for y in val_result]

# plotting a graph for both training and validation loss
plt.plot(train_iter, train_accuracy, '-o', label='Accuracy')
plt.plot(val_iter, val_accuracy, '-o', label ='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy graph')
plt.legend()
plt.show()

In [ ]:
#storing the first two items in each tuplein a separate lists
train_iter = [x[0] for x in best_result]
train_accuracy = [x[2] for x in best_result]
val_iter = [x[0] for x in val_result]
val_accuracy = [x[1] for x in val_result]

# Extract the accuracy and losses from the result
iter, accuracy, losses = zip(*result)

# plotting a graph for both training and validation loss
plt.plot(train_iter, train_accuracy, '-o', label='Training')
plt.plot(val_iter, val_accuracy, '-o', label ='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss function graph')
plt.legend()
plt.show()

In [ ]:
# saving the CNN model state in a dictionary
torch.save(model.state_dict(), 'cnn_model.pth')


In [ ]:
trainingTime

STEP 6:

In [ ]:
 # loading the saved MLP model
mlp_model = MLP(input_size, hidden_size, num_classes, activation, batch_norm, dropout, regularization).to(device)
mlp_model.load_state_dict(torch.load('mlp_model.pth'))
mlp_model.eval()

# running the testing data through the saved MLP model
test_accuracy = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = mlp_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        test_accuracy += (predicted == labels).sum().item()

# computing the test data accuracy
Accuracy = test_accuracy / total
testAccuracy = 100 * Accuracy
print(f"Test Accuracy: {testAccuracy:.2f}%")

In [ ]:
 # loading the saved CNN model
cnn_model = CNN(input_size, num_classes, activation, batch_norm, dropout, regularization).to(device)
cnn_model.load_state_dict(torch.load('cnn_model.pth'))
cnn_model.eval()

# running the testing data through the saved CNN model
test_accuracy = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        test_accuracy += (predicted == labels).sum().item()

# computing the test data accuracy
Accuracy = test_accuracy / total
testAccuracy = 100 * Accuracy
print(f"Test Accuracy: {testAccuracy:.2f}%")

Top 6 samples

In [ ]:
with torch.no_grad():
    for images, labels in test_loader:
        # Move the images and labels to the device
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn_model(images)      
        _, predicted = torch.max(outputs.data, 1) # Get the predicted classes  

        for i in range(6): # printing the top six predictions along with the true labels
            print(f"Predicted: {predicted[i]}, True Label: {labels[i]}")

        break

In [ ]:
with torch.no_grad():
    for images, labels in test_loader:
        # moving the images and labels to the device
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn_model(images)       
        _, predicted = torch.max(outputs.data, 1) # Get the predicted classes
       
        for i in range(5):  # printing the top 5 predictions along with the true labels
            print(f"Predicted: {predicted[i]}, True Label: {labels[i]}")
            plt.imshow(images[i].cpu().numpy().squeeze(), cmap='gray')
            plt.show()

        break

Confusion Matrix

In [ ]:
# get true labels
true_labels = []
for _, label in test_loader:
    true_labels += label.tolist()

In [ ]:
# make predictions using MLP model
mlp_predictions = []
mlp_model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = mlp_model(images)
        _, predicted = torch.max(outputs.data, 1)
        mlp_predictions += predicted.tolist()
# create confusion matrices
mlp_cm = confusion_matrix(true_labels, mlp_predictions)

# make predictions using CNN model
cnn_predictions = []
cnn_model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        cnn_predictions += predicted.tolist()
        
# create confusion matrices
cnn_cm = confusion_matrix(true_labels, cnn_predictions)

In [ ]:
# plotting the confusion matrices for both models
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(30, 30))

# MLP confusion matrix
sns.heatmap(mlp_cm, annot=True, fmt='g', cmap='OrRd', ax=ax1)
ax1.set_xlabel('Predicted', weight='bold')
ax1.set_ylabel('True', weight='bold')
ax1.set_title('MLP Confusion Matrix')

# CNN confusion matrix
sns.heatmap(cnn_cm, annot=True, fmt='g', cmap='OrRd', ax=ax2)
ax2.set_xlabel('Predicted', weight='bold')
ax2.set_ylabel('True', weight='bold')
ax2.set_title('CNN Confusion Matrix')

plt.tight_layout()
plt.show()


In [ ]:
# setting up the model to evaluation mode for MLP to print the precision, recall, F1 and support for the models 
mlp_model.eval()

# defining the lists to store the true and predicted labels
true_labels = []
pred_labels = []

# looping through the test data and generate predictions
with torch.no_grad():
    for images, labels in test_loader:
        # move the images and labels to the device
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        # append the true and predicted labels to the lists
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

# printing the classification report with the  precision, recall, F1 and support 
print(classification_report(true_labels, pred_labels))


In [ ]:
# settin up the model to evaluation mode for CNN to print the precision, recall, F1 and support for the models 
cnn_model.eval()

# defining the lists to store the true and predicted labels
true_labels = []
pred_labels = []

# looping through the test data and generate predictions
with torch.no_grad():
    for images, labels in test_loader:
        # move the images and labels to the device
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        # append the true and predicted labels to the lists
        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())

# print the classification report with the  precision, recall, F1 and support 
print(classification_report(true_labels, pred_labels))